In [ ]:
import pandas as pd
import numpy as np
import xarray as xr

# 20210504_munge_ll_hosp_data

The purpose of this notebook is to reformat the (local-only) hospitalization line list data in the file `data/CV19Hospital_ICU_DeID_20210414.csv`. **Important**: before committing this notebook, ensure to clear cell outputs, since these data should not be pushed to public cloud.

Right now, data have the following schema:
```csv
ID,Age,Zip Code of Residence,Hospital,Date of Admission,ICU?,Ventilator?,Date of Discharge,Discharge Status,Occupation
9999,99,99999,hosp1,2020-99-99,False,False,2020-99-99,HOSPICE - MEDICAL FACILITY 51,Retired
9999,99,99999,hosp1,2020-99-99,False,False,2020-99-99,Expired 20,Retired
9999,99,99999,hosp2,2020-99-99,False,False,2020-99-99,HOME,Resident
9999,99,99999,hosp2,2020-99-99,False,False,2020-99-99,HOME,Construction and Extraction Occupations
```

And we want to transform into something like:
```csv
date,zip_code,hosp,deceased
```

----

Convert to CSV: 
```
in2csv data/CV19Hospital_ICU_DeID_20210414.xlsx > data/CV19Hospital_ICU_DeID_20210414.csv
```

In [ ]:
raw_df = pd.read_csv('../data/CV19Hospital_ICU_DeID_20210414.csv', parse_dates=True)

Is it plausible to pull death data out of these? And fit to `Ih2D`?

In [ ]:
raw_df['Discharge Status'].unique()

Save this for later; there are way too many outcomes documented here that could be categorized as deaths.

In [ ]:
raw_df.head()

In [ ]:
df = (
    raw_df
    [['ID', 'Zip Code of Residence', 'Date of Admission', 'Date of Discharge']]
    .rename(columns={'ID': 'id', 'Zip Code of Residence': 'zip_code', 'Date of Admission': 'admission_date', 'Date of Discharge': 'discharge_date'})
    # null admission discharge dates
    .replace('No Discharge Date', np.nan)
)

Convert to datetimes:

In [ ]:
df['discharge_date'] = pd.to_datetime(df['discharge_date'], format="%Y-%m-%d")
df['admission_date'] = pd.to_datetime(df['admission_date'], format="%Y-%m-%d")

In [ ]:
df.head()

Next, use groupby objects to catalogue admission and discharge events, indexed by zip code and date:

In [ ]:
admissions = (
    df
    .rename(columns={'admission_date': 'date'})
    .groupby(['zip_code', 'date'])
    ['id']
    .count()
)
discharges = (
    df
    .rename(columns={'discharge_date': 'date'})
    .groupby(['zip_code', 'date'])
    ['id']
    .count()
)
delta = admissions.sub(discharges, fill_value=0.)
delta.head()

Calculate net change for each zip code. In theory, this should be zero:

In [24]:
delta_per_zip = delta.groupby('zip_code').sum()
# delta_per_zip
with_unacc_dc = delta_per_zip[delta_per_zip > 0].count()
unacc_dc = delta_per_zip[delta_per_zip > 0].sum().astype(int)
acc_dc = -delta[delta < 0].sum().astype(int)
print(f"There are {with_unacc_dc} zip codes with unaccounted discharges out of {len(delta_per_zip.index)} zip codes.")
print(f"{unacc_dc} unaccounted for discharges and {acc_dc} accounted for discharges.")

There are 84 zip codes with unaccounted discharges out of 509 zip codes.
217 unaccounted for discharges and 7942 accounted for discharges.


Any patterns at a glance for null discharge date rows?

In [ ]:
raw_df[df['discharge_date'].isnull()].head()

Not really, this might mean deceased, but still gives us no information on date of discharge.

What should we do about unaccounted for discharges? Possible solutions:
- Set discharge date as max date
    - Inflates Ih
- Ignore admission and discharge for `id` with unaccounted discharge
    - Deflates Ih
- Calculate the mean hospital duration and use that to extrapolate a discharge date

Will probably need to discuss with KP. Until then, take option #1, which is probably the easiest at this point, since it just means "ignore any possible missed discharge events."

In [ ]:
delta.head()

Need a continuous `date` index to assign to second MultiIndex level:

In [21]:
cont_dt = pd.date_range(
    start=delta.index.get_level_values(1).min(),
    end=delta.index.get_level_values(1).max(), 
    freq='D'
)
cont_dt

DatetimeIndex(['2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
               '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
               '2020-03-14', '2020-03-15',
               ...
               '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
               '2021-04-09', '2021-04-10', '2021-04-11', '2021-04-12',
               '2021-04-13', '2021-04-14'],
              dtype='datetime64[ns]', length=405, freq='D')

Annoyingly, `reindex` doesn't broadcast at second level in a MultiIndex:

In [ ]:
delta.reindex(cont_dt, level=1, fill_value=0., copy=True)

...but it works fine when there's only a single index:

In [ ]:
delta['10029'].reindex(cont_dt, fill_value=0., copy=True)

One workaround is to reindex direclty using a MultiIndex: https://stackoverflow.com/questions/53286882/how-to-reindex-a-multiindex-dataframe

In [ ]:
cont_midx = pd.MultiIndex.from_product(
    [delta.index.levels[0], cont_dt],
    names=['zip_code', 'date'])
ih = delta.reindex(cont_midx, fill_value=0., copy=True)
# remove 'Unknown' zip code
del ih['Unknown']

_TODO_: see if we can do this in Dask, since this workflow isn't that scalable in pandas alone.

In [ ]:
ih

## Convert to DataArray and Write to Zarr

Let's convert to an `xarray.DataArray` and write it to a zarr archive, in the same folder (`../data`):

In [ ]:
ih_da = xr.DataArray.from_series(ih)
ih_da

In [20]:
ih_da.to_netcdf('../data/CV19Hospital_ICU_DeID_20210414.nc')
!du -hs '../data/CV19Hospital_ICU_DeID_20210414.nc'

2.6M	../data/CV19Hospital_ICU_DeID_20210414.nc
